In [1]:
import numpy as np                     # numeric python lib

import matplotlib.image as mpimg       # reading images to numpy arrays
import matplotlib.pyplot as plt        # to plot any graph
import matplotlib.patches as mpatches  # to draw a circle at the mean contour

from skimage import measure            # to find shape contour
import scipy.ndimage as ndi            # to determine shape centrality

from skimage.feature import corner_harris, corner_subpix, corner_peaks, CENSURE
from scipy.signal import argrelextrema
import os
# matplotlib setup
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (6, 6)  


In [5]:
x = np.array([[[1,11,111], [2,22,222], [3,33,333]], 
              [[4,44,444], [5,55,555], [6,66,666]], 
              [[7,77,777], [8,88,888], [9,99,999]]])

# reverse the first dimension
# take the 0th element
# and take its last element
x[::-1, 0, -1]
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return [rho, phi]

In [6]:
def compute(img):
    # using image processing module of scipy to find the center of the leaf
    cy, cx = ndi.center_of_mass(img)
    contours = measure.find_contours(img, .8)

    # from which we choose the longest one
    contour = max(contours, key=len)
    
    # just calling the transformation on all pairs in the set
    polar_contour = np.array([cart2pol(x, y) for x, y in contour])
    
    contour[::,1] -= cx  # demean X
    contour[::,0] -= cy  # demean Y
    
    polar_contour = np.array([cart2pol(x, y) for x, y in contour])
    
    detector = CENSURE()
    detector.detect(img)

    coords = corner_peaks(corner_harris(img), min_distance=5)
    coords_subpix = corner_subpix(img, coords, window_size=13)
    
#     c_max_index = argrelextrema(polar_contour[::,0], np.greater, order=50)
#     c_min_index = argrelextrema(polar_contour[::,0], np.less, order=50)
    
    return polar_contour[polar_contour[:,1].argsort()]

In [7]:
for filename in os.listdir("./data/images/"):
    img = mpimg.imread('./data/images/' + filename)
    
    np.savetxt('./data/features/'+filename[:-4]+'.txt', compute(img), fmt='%f')
